In [35]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [36]:
import xdatasets as xd
import intake
import numpy as np
import geopandas as gpd
import pandas as pd
from dask.distributed import Client

In [37]:
paths = ['/home/slanglois/Downloads/hydat_basins/01/01AA002/01AA002_DrainageBasin_BassinDeDrainage.shp',
         '/home/slanglois/Downloads/hydat_basins/09/09AA003/09AA003_DrainageBasin_BassinDeDrainage.shp',
         '/home/slanglois/Downloads/hydat_basins/05/05AA002/05AA002_DrainageBasin_BassinDeDrainage.shp']

In [38]:
gdf = pd.concat([gpd.read_file(path).to_crs(4326) for path in paths]).reset_index(drop=True)
gdf

,StationNum,NameNom,Status,Etat,Area_km2,Aire_km2,Version,Date,geometry
0,01AA002,DAAQUAM (RIVIERE) EN AVAL DE LA RIVIERE SHIDGEL,discontinued,fermée,590.358,590.358,Prerelease/Préliminaire,2021-12-17,"POLYGON ((-70.23909 46.67066, -70.23945 46.670..."
1,09AA003,LINDEMAN LAKE NEAR BENNETT,discontinued,fermée,220.972,220.972,Prerelease/Préliminaire,2021-12-17,"POLYGON ((-135.00660 59.74362, -135.01203 59.7..."
2,05AA002,CROWSNEST RIVER NEAR LUNDBRECK,discontinued,fermée,676.556,676.556,Prerelease/Préliminaire,2021-12-17,"POLYGON ((-114.57621 49.73106, -114.57701 49.7..."


In [46]:
%%time

# http://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#offset-aliases

query = {
    "variables": {"era5_reanalysis_single_levels_dev": ["t2m", "tp"]},
    "space": {
        "clip": "polygon", # bbox, point or polygon
        "aggregation": True,
        "geometry": gdf,
        "unique_id": "StationNum"
    },
    "time": {
        "timestep": "D",
        "aggregation": {"tp": np.nansum, 
                        "t2m": [np.nanmax, np.nanmin]},
        
        "start": '1959-01-01',
        "end": '1960-05-31',
        "timezone": 'America/Montreal',
    },
}


xds = xd.Dataset(**query)

3it [00:02,  1.02it/s]
Processing tp: 100%|██████████████████████████████| 2/2 [00:01<00:00,  1.16it/s]


CPU times: user 9.42 s, sys: 238 ms, total: 9.66 s
Wall time: 10.1 s


In [47]:
xds.data

<xarray.Dataset>
Dimensions:     (StationNum: 3, time: 518)
Coordinates:
    longitude   (StationNum) float64 -70.26 -135.2 -114.5
    latitude    (StationNum) float64 46.53 59.77 49.62
    geom        (StationNum) int64 0 1 2
  * StationNum  (StationNum) object '01AA002' '09AA003' '05AA002'
    NameNom     (StationNum) object 'DAAQUAM (RIVIERE) EN AVAL DE LA RIVIERE ...
    Status      (StationNum) object 'discontinued' 'discontinued' 'discontinued'
    Etat        (StationNum) object 'fermée' 'fermée' 'fermée'
    Area_km2    (StationNum) float64 590.4 221.0 676.6
    Aire_km2    (StationNum) float64 590.4 221.0 676.6
    Version     (StationNum) object 'Prerelease/Préliminaire' ... 'Prerelease...
    Date        (StationNum) object '2021-12-17' '2021-12-17' '2021-12-17'
  * time        (time) datetime64[ns] 1958-12-31 1959-01-01 ... 1960-05-31
Data variables:
    t2m_nanmax  (time, StationNum) float32 257.3 247.0 276.2 ... 274.1 286.3
    t2m_nanmin  (time, StationNum) float32 255.2 245.2 274.8 ... 269.4 280.2
    tp_nansum   (time, StationNum) float32 0.0 0.0 0.0 ... 0.0009561 0.002057
Attributes:
    Conventions:               CF-1.6
    history:                   2022-11-10 02:03:41 GMT by grib_to_netcdf-2.25...
    pangeo-forge:inputs_hash:  7edb61bb3405df4b000e60bf58f3ec8ace80f6b479d9eb...
    pangeo-forge:recipe_hash:  75f0ecbd6f60ed865019e34f1daf571bc6f958d2abb07e...
    pangeo-forge:version:      0.9.4
    timezone:                  America/Montreal
    regrid_method:             conservative

In [54]:
(xds.data.hvplot(x='time', y='t2m_nanmax', grid=True, width=600, height=400, groupby='StationNum') * \
xds.data.hvplot(x='time', y='t2m_nanmin', grid=True, groupby='StationNum') + \
 gdf.hvplot(geo=True, width=600, height=400, tiles='ESRI')) \
.cols(1)

:Layout
   .DynamicMap.I :DynamicMap   [StationNum]
      :Overlay
         .Curve.I  :Curve   [time]   (t2m_nanmax)
         .Curve.II :Curve   [time]   (t2m_nanmin)
   .Overlay.I    :Overlay
      .Tiles.I    :Tiles   [x,y]
      .Polygons.I :Polygons   [Longitude,Latitude]

In [59]:
xds.data.to_dataframe().hvplot(x='time', 
                               y=['t2m_nanmax','t2m_nanmin'],
                               grid=True, 
                               width=1000,
                               height=500,
                               groupby='StationNum',
                               widget_location='bottom') 

Column
    [0] HoloViews(DynamicMap, height=500, sizing_mode='fixed', widget_location='bottom', width=1000)
    [1] Row
        [0] HSpacer()
        [1] WidgetBox
            [0] Select(margin=(20, 20, 20, 20), name='StationNum', options=['01AA002', '09AA003', ...], value='01AA002', width=250)
        [2] HSpacer()

In [110]:
ds_new

<xarray.Dataset>
Dimensions:     (time: 368184, geom: 2)
Coordinates:
  * time        (time) datetime64[ns] 1979-01-01 ... 2020-12-31T23:00:00
    longitude   (geom) float64 -70.26 -133.7
    latitude    (geom) float64 46.53 59.64
  * geom        (geom) int64 0 1
    StationNum  (geom) object '01AA002' '09AA001'
    NameNom     (geom) object 'DAAQUAM (RIVIERE) EN AVAL DE LA RIVIERE SHIDGE...
    Status      (geom) object 'discontinued' 'active'
    Etat        (geom) object 'fermée' 'en service'
    Area_km2    (geom) float64 590.4 6.889e+03
    Aire_km2    (geom) float64 590.4 6.889e+03
    Version     (geom) object 'Prerelease/Préliminaire' 'Prerelease/Prélimina...
    Date        (geom) object '2021-12-17' '2021-12-17'
Data variables:
    t2m         (time, geom) float32 dask.array<chunksize=(14880, 1), meta=np.ndarray>
    tp          (time, geom) float32 dask.array<chunksize=(14880, 1), meta=np.ndarray>
Attributes:
    institution:    ECMWF
    source:         Reanalysis
    title:          ERA5 forecasts
    regrid_method:  conservative

In [93]:
shape_bbox_indexer(ds, row.to_frame().T)

AttributeError: 'Series' object has no attribute 'unary_union'